In [1]:
""" Main Script to evaluate a trained model """
print 


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch.autograd import Variable
import numpy as np


# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import math
import config
import matplotlib.pyplot as plt 
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import utils.checkpoints as checkpoints
import adversarial_perturbations as ap
import adversarial_attacks_refactor as aar 
import spatial_transformers as st 

In [3]:
# LOAD THE PRETRAINED (DEFENDED) MODEL (NOTE: THIS HAS TO BE RESNET32)
USE_GPU = torch.cuda.is_available()
EXPERIMENT_NAME = "pgd_linf8_lr01_iter50" #INSERT EXPERIMENT NAME HERE

classifier_net, cifar_normer = cifar_loader.load_pretrained_cifar_resnet(flavor=32, use_gpu=USE_GPU, 
                                                                    return_normalizer=True) 


defended_state_dict_file = checkpoints.params_to_filename(EXPERIMENT_NAME, 'resnet32')[-1]
print "Loading %s params into resnet32" % defended_state_dict_file 

classifier_net = checkpoints.load_state_dict_from_filename(defended_state_dict_file, classifier_net)

Loading pgd_linf8_lr01_iter50.resnet32.000032.path.tar params into resnet32


In [4]:
# LOAD THE EVALUATION DATASET 
val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          batch_size=128, use_gpu=USE_GPU, 
                                         shuffle=True)


Files already downloaded and verified


In [5]:
reload(adveval)
# Build various attacks to test 
L_INF_BOUND = 8.0/255.0 

##############################################################################
#                                                                            #
#        PGD LINFINITY                                                       #
#                                                                            #
##############################################################################

    
def build_delta_pgd_eval(iter_no):
    # UNSIGNED PGD ATTACK 
    delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                                  ap.PerturbationParameters(lp_style='inf',
                                                            lp_bound=L_INF_BOUND,
                                                            use_gpu=USE_GPU))
    attack_loss = plf.VanillaXentropy(classifier_net, cifar_normer)
    pgd_attack = aar.PGD(classifier_net, cifar_normer, delta_threat, attack_loss, use_gpu=USE_GPU)
    optimizer = optim.Adam
    optimizer_kwargs = {'lr': 0.01}
    pgd_kwargs = {'num_iterations': iter_no, 
                  'signed': False, 
                  'optimizer': optimizer, 
                  'optimizer_kwargs': optimizer_kwargs, 
                  'verbose': False}      
    params = advtrain.AdversarialAttackParameters(pgd_attack, 1.0, 
                                       attack_specific_params={'attack_kwargs': pgd_kwargs})
    to_eval= {'top1': 'top1',
              'lpips': 'avg_successful_lpips'}
    
    eval_result = adveval.EvaluationResult(params, classifier_net, cifar_normer, 
                                           to_eval=to_eval, use_gpu=USE_GPU)
    return eval_result     

##############################################################################
#                                                                            #
#      PGD FLOW                                                              #
#                                                                            #
##############################################################################

def build_stadv_pgd_eval(iter_no):
    # STAdv style attack 
    flow_threat = ap.ThreatModel(ap.ParameterizedXformAdv, 
                                 ap.PerturbationParameters(lp_style='inf', 
                                                           lp_bound=2,
                                                           xform_class=st.FullSpatial, 
                                                           use_gpu=USE_GPU))
    adv_loss = lf.CWLossF6(classifier_net, cifar_normer)
    st_loss = lf.PerturbationNormLoss(lp=2)
    
    loss_fxn = lf.RegularizedLoss({'adv': adv_loss, 'st':st_loss}, 
                                  {'adv': 1.0, 'st': 0.05},
                                  negate=True)  
    
    optimizer = optim.Adam
    optimizer_kwargs = {'lr': 0.001}
    pgd_attack = aar.PGD(classifier_net, cifar_normer, flow_threat, loss_fxn, use_gpu=USE_GPU)
    pgd_kwargs = {'num_iterations': iter_no,  
                  'signed': False, 
                  'optimizer': optimizer, 
                  'optimizer_kwargs': optimizer_kwargs, 
                  'verbose': False}    


    params = advtrain.AdversarialAttackParameters(pgd_attack, 1.0, 
                                       attack_specific_params={'attack_kwargs': pgd_kwargs})
    to_eval= {'top1': 'top1',
              'lpips': 'avg_successful_lpips'}    
    eval_result = adveval.EvaluationResult(params, classifier_net, cifar_normer, 
                                           to_eval=to_eval, use_gpu=USE_GPU)
    return eval_result  




In [6]:
def verify_eval(eval_obj, num=4):
    examples, labels = next(iter(val_loader))
    examples = examples.cuda()
    labels = labels.cuda() 
    attack_out = eval_obj.attack_params.attack(examples, labels)
    img_utils.show_images([attack_out[0][:num], examples[:num], 
                           attack_out[0][:num] - examples[:num] + 0.5])

In [7]:
# BUILD ATTACK ENSEMBLE 
ensemble = {'pgd_%03d' % i : build_delta_pgd_eval(i) for i in [1, 5, 10, 20, 50, 100, 200]}
            
eval_obj = adveval.AdversarialEvaluation(classifier_net, cifar_normer, use_gpu=USE_GPU)
eval_output = eval_obj.evaluate_ensemble(val_loader, ensemble, verbose=True, num_minibatches=3)


Starting minibatch 0...
	 (mb: 0) evaluating pgd_010...
	 (mb: 0) evaluating pgd_001...
	 (mb: 0) evaluating pgd_005...
	 (mb: 0) evaluating pgd_100...
	 (mb: 0) evaluating pgd_020...
	 (mb: 0) evaluating pgd_200...
	 (mb: 0) evaluating pgd_050...
	 (mb: 0) evaluating ground...
Starting minibatch 1...
	 (mb: 1) evaluating pgd_010...
	 (mb: 1) evaluating pgd_001...
	 (mb: 1) evaluating pgd_005...
	 (mb: 1) evaluating pgd_100...
	 (mb: 1) evaluating pgd_020...
	 (mb: 1) evaluating pgd_200...
	 (mb: 1) evaluating pgd_050...
	 (mb: 1) evaluating ground...
Starting minibatch 2...
	 (mb: 2) evaluating pgd_010...
	 (mb: 2) evaluating pgd_001...
	 (mb: 2) evaluating pgd_005...
	 (mb: 2) evaluating pgd_100...
	 (mb: 2) evaluating pgd_020...
	 (mb: 2) evaluating pgd_200...
	 (mb: 2) evaluating pgd_050...
	 (mb: 2) evaluating ground...


In [9]:
for k, v in sorted(eval_output.items(), key=lambda d: d[0]):
    if k != 'ground':
        print k, v.results['top1'].avg, v.results['lpips']
    else:
        print k, v.results['top1'].avg

ground 0.830729166667
pgd_001 0.700520833333 0.000622766103166
pgd_005 0.416666666667 0.00538342815036
pgd_010 0.390625 0.00547844352057
pgd_020 0.3828125 0.00541843157262
pgd_050 0.380208333333 0.00545579619849
pgd_100 0.380208333333 0.00550063445878
pgd_200 0.377604166667 0.00547154405369
